### Manipulation of PEAKS de novo results of Trocas 8 (low water, November 2019) non-incubation samples LC-MS/MS data using python.

Starting with:

    PEAKS de novo results (.csv) of PTM-optimized sequencing >50% ALC

Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the TROCAS # (e.g., 675) for another
#### 2. Change output file name in *IN 6*, *IN 7*, *IN 8*

In [1]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [2]:
    cd /home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/T8-June2021-UWPR-DN50_10lgP15/678_Trocas8_June2021_DENOVO_105/

/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/T8-June2021-UWPR-DN50_10lgP15/678_Trocas8_June2021_DENOVO_105


In [3]:
# read the CSVs into a dataframe using the read_csv function and call 'peaks'

peaks678 = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/T8-June2021-UWPR-DN50_10lgP15/678_Trocas8_June2021_DENOVO_105/678_Trocas8_June2021_DENOVO_105_DN50.csv")

print("# redundant Peaks peptides >50% ALC in combined dataframe:", len(peaks678))

print(peaks678.columns)

# These columns mess things up- get rid of them

del peaks678['Fraction']
del peaks678['Scan']
del peaks678['Source File']
del peaks678['Tag Length']
del peaks678['PTM']
del peaks678['tag (>=0%)']
del peaks678['mode']
del peaks678['local confidence (%)']


columns = ['Peptide', 'ALC', 'length', 'm/z', 'z', 'RT', 'Area',
       'Mass', 'ppm']

peaks678.columns = columns

mean_len = peaks678['length'].mean()
print(mean_len)

# look at the dataframe
peaks678.head()

# redundant Peaks peptides >50% ALC in combined dataframe: 104
Index(['Fraction', 'Scan', 'Source File', 'Peptide', 'Tag Length', 'ALC (%)',
       'length', 'm/z', 'z', 'RT', 'Area', 'Mass', 'ppm', 'PTM',
       'local confidence (%)', 'tag (>=0%)', 'mode'],
      dtype='object')
9.067307692307692


,Peptide,ALC,length,m/z,z,RT,Area,Mass,ppm
0,LSSPATLNSR,98,10,523.2846,2,49.38,6730000.0,1044.5564,-1.7
1,LSSPATLNSR,98,10,523.2844,2,51.40,6730000.0,1044.5564,-2.1
2,LSSPATLNSR,98,10,523.2852,2,49.89,6730000.0,1044.5564,-0.6
3,LSSPATLNSR,98,10,523.2852,2,50.40,6730000.0,1044.5564,-0.5
4,LSSPATLNSR,98,10,523.2856,2,50.90,6730000.0,1044.5564,0.3


The peptide column has the masses of modifications (e.g., 57.02 Da for carbamidomethylation of cysteine). We want to make new columns with all that information and make a column with only the 'stripped' peptide sequence that's just amino acids - this we can then align against other sequences, for example.

Modified residues were allowed for:

    fixed carbamodimethylation of cysteine 57.021464 C
    varialbe oxidation of methionine: 15.9949 M
    variable deamidation of asparagine, glumatine: 0.984016 NQ

We'll then write this manipulated dataframe to a new file.

In [5]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaks678['A'] = peaks678['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaks678['C'] = peaks678['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaks678['D'] = peaks678['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaks678['E'] = peaks678['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaks678['F'] = peaks678['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaks678['G'] = peaks678['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaks678['H'] = peaks678['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaks678 output, there will be no isoleucines (they're lumped in with leucines)
peaks678['I'] = peaks678['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaks678['K'] = peaks678['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaks678['L'] = peaks678['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaks678['M'] = peaks678['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaks678['N'] = peaks678['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaks678['P'] = peaks678['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaks678['Q'] = peaks678['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaks678['R'] = peaks678['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaks678['S'] = peaks678['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaks678['T'] = peaks678['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaks678['V'] = peaks678['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaks678['W'] = peaks678['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaks678['Y'] = peaks678['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaks678['c-carb'] = peaks678['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaks678['m-oxid'] = peaks678['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaks678['n-deam'] = peaks678['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of deamidated Q's in each peptide
peaks678['q-deam'] = peaks678['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# create a column with 'stripped' peptide sequences using strip
peaks678['stripped_peptide'] = peaks678['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaks678['stripped_length'] = peaks678['stripped_peptide'].apply(len)

# total the number of modifications in sequence
peaks678['ptm-total'] = peaks678['c-carb'] + peaks678['m-oxid'] + peaks678['n-deam'] + peaks678['q-deam']

# calculate NAAF numerator for each peptide k
peaks678['NAAF_num.'] = peaks678['Area'] / peaks678['stripped_length']

# write modified dataframe to new txt file, same name + 'stripped'
peaks678.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDN/678_CV_surf_GF75_DN/678_CV_surf_GF75_DN50.csv")

# check out the results
peaks678.head()

,Peptide,ALC,length,m/z,z,RT,Area,Mass,ppm,A,...,W,Y,c-carb,m-oxid,n-deam,q-deam,stripped_peptide,stripped_length,ptm-total,NAAF_num.
0,LSSPATLNSR,98,10,523.2846,2,49.38,6730000.0,1044.5564,-1.7,1,...,0,0,0,0,0,0,LSSPATLNSR,10,0,673000.0
1,LSSPATLNSR,98,10,523.2844,2,51.40,6730000.0,1044.5564,-2.1,1,...,0,0,0,0,0,0,LSSPATLNSR,10,0,673000.0
2,LSSPATLNSR,98,10,523.2852,2,49.89,6730000.0,1044.5564,-0.6,1,...,0,0,0,0,0,0,LSSPATLNSR,10,0,673000.0
3,LSSPATLNSR,98,10,523.2852,2,50.40,6730000.0,1044.5564,-0.5,1,...,0,0,0,0,0,0,LSSPATLNSR,10,0,673000.0
4,LSSPATLNSR,98,10,523.2856,2,50.90,6730000.0,1044.5564,0.3,1,...,0,0,0,0,0,0,LSSPATLNSR,10,0,673000.0


In [6]:
# keep only stripped peptide I/L and NAAF
dn_678 = peaks678[['stripped_peptide', 'Area', 'NAAF_num.']]

dn_678.set_index('stripped_peptide')

# write modified dataframe to new txt file
dn_678.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/I-L_NAAFs/678_CV_surf_GF75_DN50_ILnaafs.csv")

dn_678.head()

,stripped_peptide,Area,NAAF_num.
0,LSSPATLNSR,6730000.0,673000.0
1,LSSPATLNSR,6730000.0,673000.0
2,LSSPATLNSR,6730000.0,673000.0
3,LSSPATLNSR,6730000.0,673000.0
4,LSSPATLNSR,6730000.0,673000.0


In [7]:
# made a new dataframe that contains the sums of certain columns 
# in the stripped peptide dataframe above (for >50% ALC)

index = ['sample total']

data = {'A': peaks678['A'].sum(),
        'C': peaks678['C'].sum(),
        'D': peaks678['D'].sum(),
        'E': peaks678['E'].sum(),
        'F': peaks678['F'].sum(),
        'G': peaks678['G'].sum(),
        'H': peaks678['H'].sum(),
        'I': peaks678['I'].sum(),
        'K': peaks678['K'].sum(),
        'L': peaks678['L'].sum(),
        'M': peaks678['M'].sum(),
        'N': peaks678['N'].sum(),
        'P': peaks678['P'].sum(),
        'Q': peaks678['Q'].sum(),
        'R': peaks678['R'].sum(),
        'S': peaks678['S'].sum(),
        'T': peaks678['T'].sum(),
        'V': peaks678['V'].sum(),
        'W': peaks678['W'].sum(),
        'Y': peaks678['Y'].sum(),
        'c-carb': peaks678['c-carb'].sum(),
        'm-oxid': peaks678['m-oxid'].sum(),
        'n-deam': peaks678['n-deam'].sum(),
        'q-deam': peaks678['q-deam'].sum(),
        'Total area': peaks678['Area'].sum(),
        'Total length': peaks678['stripped_length'].sum()
       }

totalpeaks678 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                            'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                            'c-carb', 'm-oxid', 'n-deam', 'q-deam', \
                                            'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaks678['% C w/ carb'] = totalpeaks678['c-carb'] / totalpeaks678['C'] 

# calculate percentage of M's that are oxidized
totalpeaks678['% M w/ oxid'] = totalpeaks678['m-oxid'] / totalpeaks678['M'] 

# calculate percentage of N's that are deamidated
totalpeaks678['% N w/ deam'] = totalpeaks678['n-deam'] / totalpeaks678['N'] 

# calculate percentage of N's that are deamidated
totalpeaks678['% Q w/ deam'] = totalpeaks678['q-deam'] / totalpeaks678['Q'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaks678['NAAF denom.'] = totalpeaks678['Total area'] / totalpeaks678['Total length']

# write modified dataframe to new txt file
totalpeaks678.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDN/678_CV_surf_GF75_DN/678_CV_surf_GF75_DN50_totals.csv")

totalpeaks678.head()

,A,C,D,E,F,G,H,I,K,L,...,m-oxid,n-deam,q-deam,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF denom.
sample total,60,7,7,8,10,15,51,0,23,143,...,11,1,0,2.380996e+08,924,1.0,0.203704,0.038462,0.0,257683.51842


In [8]:
# use the calculated NAAF factor (in totalpeaks dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

# don't have to worry here about DECOY hits messing with Area totals
# but we would with Comet results

NAAF50 = 257683.51842

# use NAAF >50% ALC to get NAAF factor
peaks678['NAAF factor'] = (peaks678['NAAF_num.'])/NAAF50

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaksAAPTM_678 = peaks678[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'K', 'I', 'L', \
                                'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', \
                                'n-deam', 'q-deam']].copy()

# multiply the NAAF50 factor by the AA total to normalize its abundance by peak area and peptide length

peaksAAPTM_678['A-NAAF50'] = peaksAAPTM_678['A'] * peaks678['NAAF factor']
peaksAAPTM_678['C-NAAF50'] = peaksAAPTM_678['C'] * peaks678['NAAF factor']
peaksAAPTM_678['D-NAAF50'] = peaksAAPTM_678['D'] * peaks678['NAAF factor']
peaksAAPTM_678['E-NAAF50'] = peaksAAPTM_678['E'] * peaks678['NAAF factor']
peaksAAPTM_678['F-NAAF50'] = peaksAAPTM_678['F'] * peaks678['NAAF factor']
peaksAAPTM_678['G-NAAF50'] = peaksAAPTM_678['G'] * peaks678['NAAF factor']
peaksAAPTM_678['H-NAAF50'] = peaksAAPTM_678['H'] * peaks678['NAAF factor']
peaksAAPTM_678['I-NAAF50'] = peaksAAPTM_678['I'] * peaks678['NAAF factor']
peaksAAPTM_678['K-NAAF50'] = peaksAAPTM_678['K'] * peaks678['NAAF factor']
peaksAAPTM_678['L-NAAF50'] = peaksAAPTM_678['L'] * peaks678['NAAF factor']
peaksAAPTM_678['M-NAAF50'] = peaksAAPTM_678['M'] * peaks678['NAAF factor']
peaksAAPTM_678['N-NAAF50'] = peaksAAPTM_678['N'] * peaks678['NAAF factor']
peaksAAPTM_678['P-NAAF50'] = peaksAAPTM_678['P'] * peaks678['NAAF factor']
peaksAAPTM_678['Q-NAAF50'] = peaksAAPTM_678['Q'] * peaks678['NAAF factor']
peaksAAPTM_678['R-NAAF50'] = peaksAAPTM_678['R'] * peaks678['NAAF factor']
peaksAAPTM_678['S-NAAF50'] = peaksAAPTM_678['S'] * peaks678['NAAF factor']
peaksAAPTM_678['T-NAAF50'] = peaksAAPTM_678['T'] * peaks678['NAAF factor']
peaksAAPTM_678['V-NAAF50'] = peaksAAPTM_678['V'] * peaks678['NAAF factor']
peaksAAPTM_678['W-NAAF50'] = peaksAAPTM_678['W'] * peaks678['NAAF factor']
peaksAAPTM_678['Y-NAAF50'] = peaksAAPTM_678['Y'] * peaks678['NAAF factor']

# multiply the NAAF50 factor by the PTMs normalize its abundance by peak area and peptide length

peaksAAPTM_678['ccarb-NAAF50'] = peaksAAPTM_678['c-carb'] * peaksAAPTM_678['NAAF factor']
peaksAAPTM_678['moxid-NAAF50'] = peaksAAPTM_678['m-oxid'] * peaksAAPTM_678['NAAF factor']
peaksAAPTM_678['ndeam-NAAF50'] = peaksAAPTM_678['n-deam'] * peaksAAPTM_678['NAAF factor']
peaksAAPTM_678['qdeam-NAAF50'] = peaksAAPTM_678['q-deam'] * peaksAAPTM_678['NAAF factor']


# write the dataframe to a new csv
peaksAAPTM_678.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDN/678_CV_surf_GF75_DN/678_CV_surf_GF75_DN50_naaf.csv")

peaksAAPTM_678.head()

,stripped_peptide,NAAF factor,A,C,D,E,F,G,H,K,...,R-NAAF50,S-NAAF50,T-NAAF50,V-NAAF50,W-NAAF50,Y-NAAF50,ccarb-NAAF50,moxid-NAAF50,ndeam-NAAF50,qdeam-NAAF50
0,LSSPATLNSR,2.611731,1,0,0,0,0,0,0,0,...,2.611731,7.835193,2.611731,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,LSSPATLNSR,2.611731,1,0,0,0,0,0,0,0,...,2.611731,7.835193,2.611731,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,LSSPATLNSR,2.611731,1,0,0,0,0,0,0,0,...,2.611731,7.835193,2.611731,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,LSSPATLNSR,2.611731,1,0,0,0,0,0,0,0,...,2.611731,7.835193,2.611731,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,LSSPATLNSR,2.611731,1,0,0,0,0,0,0,0,...,2.611731,7.835193,2.611731,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# made a dataframe that's the sum of NAAF corrected AAs and PTMs

index = ['sample total']

data = {'NAAF': peaksAAPTM_678['NAAF factor'].sum(),
        'A': peaksAAPTM_678['A-NAAF50'].sum(),
        'C': peaksAAPTM_678['C-NAAF50'].sum(),
        'D': peaksAAPTM_678['D-NAAF50'].sum(),
        'E': peaksAAPTM_678['E-NAAF50'].sum(),
        'F': peaksAAPTM_678['F-NAAF50'].sum(),
        'G': peaksAAPTM_678['G-NAAF50'].sum(),
        'H': peaksAAPTM_678['H-NAAF50'].sum(),
        'I': peaksAAPTM_678['I-NAAF50'].sum(),
        'K': peaksAAPTM_678['K-NAAF50'].sum(),
        'L': peaksAAPTM_678['L-NAAF50'].sum(),
        'M': peaksAAPTM_678['M-NAAF50'].sum(),
        'N': peaksAAPTM_678['N-NAAF50'].sum(),
        'P': peaksAAPTM_678['P-NAAF50'].sum(),
        'Q': peaksAAPTM_678['Q-NAAF50'].sum(),
        'R': peaksAAPTM_678['R-NAAF50'].sum(),
        'S': peaksAAPTM_678['S-NAAF50'].sum(),
        'T': peaksAAPTM_678['T-NAAF50'].sum(),
        'V': peaksAAPTM_678['V-NAAF50'].sum(),
        'W': peaksAAPTM_678['W-NAAF50'].sum(),
        'Y': peaksAAPTM_678['Y-NAAF50'].sum(),
        'c-carb': peaksAAPTM_678['ccarb-NAAF50'].sum(),
        'm-oxid': peaksAAPTM_678['moxid-NAAF50'].sum(),
        'n-deam': peaksAAPTM_678['ndeam-NAAF50'].sum(),
        'q-deam': peaksAAPTM_678['qdeam-NAAF50'].sum(),
       }

totalpeaks50_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', \
                                           'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', \
                                           'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', 'q-deam'], index=index)

# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaks50_NAAF['% C w/ carb.'] = totalpeaks50_NAAF['c-carb'] / totalpeaks50_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaks50_NAAF['% M w/ oxid'] = totalpeaks50_NAAF['m-oxid'] / totalpeaks50_NAAF['M'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaks50_NAAF['% N w/ deam'] = totalpeaks50_NAAF['n-deam'] / totalpeaks50_NAAF['N'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaks50_NAAF['% Q w/ deam'] = totalpeaks50_NAAF['q-deam'] / totalpeaks50_NAAF['Q'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaks50_NAAF['NAAF check'] = totalpeaks50_NAAF['NAAF'] / 257683.51842

# write modified dataframe to new txt file, same name + totals
totalpeaks50_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDN/678_CV_surf_GF75_DN/678_CV_surf_GF75_DN50_naaf_totals.csv")

totalpeaks50_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,Y,c-carb,m-oxid,n-deam,q-deam,% C w/ carb.,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF check
sample total,107.31664,95.357765,0.206582,0.7581,0.045119,0.142702,3.914519,0.956444,0.0,1.873303,...,0.575226,0.206582,1.001393,0.015699,0.0,1.0,0.506881,0.000486,0.0,0.000416


## Export stripped peptides >50% ALC

In [10]:
###### keep only stripped peptide column 
pep50 = peaks678[["stripped_peptide"]]

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing

pep50.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDN/678_CV_surf_GF75_DN/678_CV_surf_GF75_DN50_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDN/678_CV_surf_GF75_DN/678_CV_surf_GF75_DN50_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDN/678_CV_surf_GF75_DN/678_CV_surf_GF75_DN50_stripped_peptides.fas

# removing redundancy
peaks50dedup = pd.DataFrame.drop_duplicates(pep50)

print("# redundant stripped Peaks peptides >50% ALC", len(pep50))
print("# nonredundant stripped Peaks peptides >50% ALC", len(peaks50dedup))
print("average peptide length Peaks peptides >50% ALC", peaks678['stripped_length'].mean())

# count all unique peptide (modified peptides included)
# keep only peptide column >50% ALC
pep50m = peaks678[["Peptide"]]

# deduplicate
pep50mdedup = pd.DataFrame.drop_duplicates(pep50m)

print("# redundant Peaks peptides >50% ALC", len(pep50m))
print("# nonredundant Peaks peptides", len(pep50mdedup))

# check
pep50.head()

# redundant stripped Peaks peptides >50% ALC 104
# nonredundant stripped Peaks peptides >50% ALC 43
average peptide length Peaks peptides >50% ALC 8.884615384615385
# redundant Peaks peptides >50% ALC 104
# nonredundant Peaks peptides 43


,stripped_peptide
0,LSSPATLNSR
1,LSSPATLNSR
2,LSSPATLNSR
3,LSSPATLNSR
4,LSSPATLNSR
